In [35]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
import env
from scipy.stats import pearsonr

import wrangle_zillow

#import extra classes for modeling 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV

df = wrangle_zillow.acquire()

pd.options.display.max_rows = 100 

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

In [36]:
df.shape

(77380, 68)

In [37]:
df = wrangle_zillow.prepare_data(df)

In [38]:
df.shape

(50616, 40)

In [39]:
train, validate, test = wrangle_zillow.split_zillow_data(df)

### Filtering complete, started with 77380 rows, end 50616 rows

In [40]:
#drop columns 
train.drop(columns=['bathroomcnt', 'yearbuilt', 'finishedsquarefeet12', 'bedroomcnt', 'calculatedbathnbr', 'fullbathcnt', 'latitude', 'longitude', 'lotsizesquarefeet', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcity', 'regionidcounty', 'regionidzip', 'roomcnt', 'assessmentyear', 'censustractandblock'], inplace=True)

In [41]:
columns_to_scale=['calculatedfinishedsquarefeet', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount', 'age', 'Sqft_age', 'Sqft_structuretax',
       'Sqft_taxvalue', 'Sqft_landtax', 'Sqft_taxamount', 'age_structuretax',
       'age_taxvalue', 'age_landtax', 'age_taxamount', 'Structuretax_taxvalue',
       'Structuretax_landtax', 'Landtax_taxamount', 'structuretax_taxamount',
       'taxvalue_landtax', 'taxvalue_taxamount']

In [42]:
train_scaled = train.copy()
validate_scaled = validate.copy()
test_scaled = test.copy()

In [43]:
scaler = MinMaxScaler()
scaler.fit(train[columns_to_scale])

MinMaxScaler()

In [44]:
train_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(train[columns_to_scale]),
                                                  columns=train[columns_to_scale].columns.values).set_index([train.index.values])

In [45]:
validate_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(validate[columns_to_scale]),
                                                  columns=validate[columns_to_scale].columns.values).set_index([validate.index.values])

In [46]:
test_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(test[columns_to_scale]),
                                                 columns=test[columns_to_scale].columns.values).set_index([test.index.values])

In [47]:
train_scaled.columns

Index(['calculatedfinishedsquarefeet', 'fips', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount', 'logerror',
       'age', 'Sqft_age', 'Sqft_structuretax', 'Sqft_taxvalue', 'Sqft_landtax',
       'Sqft_taxamount', 'age_structuretax', 'age_taxvalue', 'age_landtax',
       'age_taxamount', 'Structuretax_taxvalue', 'Structuretax_landtax',
       'Landtax_taxamount', 'structuretax_taxamount', 'taxvalue_landtax',
       'taxvalue_taxamount'],
      dtype='object')

In [48]:
X_train = train_scaled[['Sqft_age', 'calculatedfinishedsquarefeet', 'taxamount', 'Sqft_structuretax', 'Sqft_taxvalue', 'Sqft_landtax',
       'Sqft_taxamount', 'age_structuretax', 'age_taxvalue', 'age_landtax',
       'age_taxamount', 'Structuretax_taxvalue', 'Structuretax_landtax',
       'Landtax_taxamount', 'structuretax_taxamount', 'taxvalue_landtax',
       'taxvalue_taxamount']]
y_train = train[['logerror']]

In [49]:
X_validate = validate_scaled[['Sqft_age', 'Sqft_structuretax', 'Sqft_taxvalue', 'Sqft_landtax',
       'Sqft_taxamount', 'age_structuretax', 'age_taxvalue', 'age_landtax',
       'age_taxamount', 'Structuretax_taxvalue', 'Structuretax_landtax',
       'Landtax_taxamount', 'structuretax_taxamount', 'taxvalue_landtax',
       'taxvalue_taxamount']]
y_validate = validate[['logerror']]

In [50]:
X_test = test_scaled[['Sqft_age', 'Sqft_structuretax', 'Sqft_taxvalue', 'Sqft_landtax',
       'Sqft_taxamount', 'age_structuretax', 'age_taxvalue', 'age_landtax',
       'age_taxamount', 'Structuretax_taxvalue', 'Structuretax_landtax',
       'Landtax_taxamount', 'structuretax_taxamount', 'taxvalue_landtax',
       'taxvalue_taxamount']]
y_test = test[['logerror']]

In [ ]:
y_train.mean()

In [ ]:
y_train.mean()

In [ ]:
#trying to beat the median 
y_train.median()

In [51]:
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor 

In [52]:
from sklearn.feature_selection import RFE

In [53]:
def select_rfe (X_df, y_df, n = 1, model = LinearRegression(normalize=True), rank = False):
    '''
    Takes in the predictors, the target, and the number of features to select (k) ,
    and returns the names of the top k selected features based on the Recursive Feature Elimination (RFE)
    
    X_df : the predictors
    y_df : the target
    n_features : the number of features to select (k)
    method : LinearRegression, LassoLars, TweedieRegressor
    Example
    select_rfe(X_train_scaled, y_train, 2, LinearRegression())
    '''
    
    rfe = RFE(estimator=model, n_features_to_select= n)
    rfe.fit_transform(X_df, y_df)
    mask = rfe.get_support()
    rfe_feature = X_df.iloc[:,mask].columns.tolist()
    # check if rank=True
    if rank == True:
        # get the ranks
        var_ranks = rfe.ranking_
        # get the variable names
        var_names = X_df.columns.tolist()
        # combine ranks and names into a df for clean viewing
        rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
        # sort the df by rank
        rfe_ranks_df = rfe_ranks_df.sort_values('Rank')
        # print DataFrame of rankings
    return rfe_feature, rfe_ranks_df


In [54]:
_, rank_features = select_rfe(X_train, y_train, n = 9, rank=True)
rank_features

,Var,Rank
16,taxvalue_taxamount,1
14,structuretax_taxamount,1
3,Sqft_structuretax,1
13,Landtax_taxamount,1
12,Structuretax_landtax,1
6,Sqft_taxamount,1
7,age_structuretax,1
15,taxvalue_landtax,1
10,age_taxamount,1
1,calculatedfinishedsquarefeet,2


In [ ]:
#Create my X and y variables for quick reference 
X = X_train.copy()
y = y_train.copy()

In [ ]:
#Create A and B variables for quick reference 
A = X_validate.copy()
B = y_validate.copy()

In [ ]:
#Create a baseline 
baseline = y_train.median()
baseline

In [ ]:
#Create dataframe with target variable and my baseline 
Ventura_df = pd.DataFrame(y)

In [ ]:
#Create dataframe with validate target variable 
Ventura_validate_df = pd.DataFrame(B)

In [ ]:
#add a column for baseline
Ventura_df['baseline'] = baseline

In [ ]:
#add a column for baseline residual, and baseline residual squared
Ventura_df['baseline_residual'] = Ventura_df.logerror - Ventura_df.baseline
Ventura_df['baseline_residual^2'] = Ventura_df.baseline_residual**2

In [ ]:
#Set the linear regression model 
lm = LinearRegression()

In [ ]:
#fit the model 
lm.fit(X_train, y_train)

In [ ]:
#print the coefficients 
lm.coef_[0]

In [ ]:
#Create a dataframe with my coefficients and x columns 
cdf = pd.DataFrame(X.columns)

In [ ]:
#create predictions
predictions = lm.predict(X_train)

In [ ]:
#create a column for predictions 
Ventura_df['yhat'] = lm.predict(X_train)

In [ ]:
X_train.columns

In [ ]:
#create validate predictions 
predictions_validate = lm.predict(X_validate)

In [ ]:
Ventura_validate_df['yhat_validate'] = lm.predict(X_validate)


In [ ]:
#print metrics
mae = sklearn.metrics.mean_absolute_error(y_train, predictions)
mse = sklearn.metrics.mean_squared_error(y_train, predictions)
rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_train, predictions))
ev = sklearn.metrics.explained_variance_score(y_train, predictions)
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Explained Variance: {ev}')

In [ ]:
RMSE2 = sklearn.metrics.mean_squared_error(y_train, Ventura_df.yhat, squared = False)
RMSE2_baseline = sklearn.metrics.mean_squared_error(y_train, Ventura_df.baseline, squared=False)

print("RMSE", RMSE2) 
print("RMSE baseline" ,RMSE2_baseline)  

In [ ]:
RMSE_train = sklearn.metrics.mean_squared_error(y_train, Ventura_df.yhat, squared = False)
RMSE_validate = sklearn.metrics.mean_squared_error(y_validate, Ventura_validate_df.yhat_validate, squared = False)

print("RMSE In Sample", RMSE_train) 
print("RMSE out of Sample" ,RMSE_validate)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
X_train = X_train.copy()
X_train = X_train.reshape(-1, 1)
X_validate = X_validate.copy()
X_validate = X_validate.reshape(-1, 1)
X_test = X_test.copy()
X_test = X_test.reshape(-1, 1)

In [ ]:
pf = PolynomialFeatures(degree=2)
X_train_degree2 = pf.fit_transform(X_train)

X_validate_degree2 = pf.transform(X_validate)
X_test_degree2 = pf.transform(X_test)

In [ ]:
lm2 = LinearRegression(normalize=True)
lm2.fit(X_train_degree2, y_train)

y_train['logerror_pred'] = lm2.predict(X_train_degree2)
y_validate['logerror_pred'] = lm2.predict(X_validate_degree2)

In [ ]:
pd.options.display.float_format = '{:.6f}'.format

model_results = pd.concat([
    y_train.apply(lambda col: mean_squared_error(y_train, col)**(1/2)).rename('train'),
    y_validate.apply(lambda col: mean_squared_error(y_validate, col)**(1/2)).rename('validate')
], axis=1)
model_results.sort_values('validate')